In [2]:
import numpy as np

def normalize(p):
    """Normalize a non-negative vector to a probability distribution."""
    p = np.array(p, dtype=float)
    if np.any(p < 0):
        raise ValueError("Probabilities must be non-negative.")
    Z = p.sum()
    if Z == 0:
        raise ValueError("Sum of probabilities is zero.")
    return p / Z



In [3]:
def H_shannon(p):
    """Shannon entropy in bits."""
    p = normalize(p)
    # avoid log(0) by masking zero entries
    mask = p > 0
    return -np.sum(p[mask] * np.log2(p[mask]))

def H_min(p):
    """Min-entropy in bits: -log2(max_x p_x)."""
    p = normalize(p)
    return -np.log2(np.max(p))

def H_max(p):
    """Max-entropy (Renner-style) in bits: 2 * log2(sum_x sqrt(p_x))."""
    p = normalize(p)
    return 2 * np.log2(np.sum(np.sqrt(p)))


In [4]:
# Example 1: fair coin
p_fair = [0.5, 0.5]

print("Fair coin:")
print("H_shannon =", H_shannon(p_fair))
print("H_min     =", H_min(p_fair))
print("H_max     =", H_max(p_fair))

# Example 2: biased coin
p_biased = [0.9, 0.1]

print("\nBiased coin:")
print("H_shannon =", H_shannon(p_biased))
print("H_min     =", H_min(p_biased))
print("H_max     =", H_max(p_biased))

# Example 3: four-outcome distribution
p_four = [0.5, 0.2, 0.2, 0.1]

print("\nFour-outcome example:")
print("H_shannon =", H_shannon(p_four))
print("H_min     =", H_min(p_four))
print("H_max     =", H_max(p_four))


Fair coin:
H_shannon = 1.0
H_min     = 1.0
H_max     = 1.0000000000000002

Biased coin:
H_shannon = 0.4689955935892812
H_min     = 0.15200309344504995
H_max     = 0.6780719051126377

Four-outcome example:
H_shannon = 1.7609640474436812
H_min     = 0.9999999999999997
H_max     = 1.8788469835018302


In [5]:
def H_min_smooth_toy(p, eps):
    """
    Toy version of smooth min-entropy.

    Idea: we allow ourselves to discard events whose total probability is up to eps,
    starting from the least likely outcomes. This is NOT the formal definition, but
    gives some intuition for how smoothing can increase H_min.
    """
    p = normalize(p)
    # sort probabilities ascending
    p_sorted = np.sort(p)
    remaining = 1.0
    i = 0
    # remove small probabilities until we've removed at most eps
    to_remove = eps
    while i < len(p_sorted) and to_remove > 0:
        if p_sorted[i] <= to_remove:
            to_remove -= p_sorted[i]
            p_sorted[i] = 0.0
        else:
            # partially reduce this probability
            p_sorted[i] -= to_remove
            to_remove = 0.0
        i += 1

    # renormalize the trimmed distribution
    p_trimmed = normalize(p_sorted)
    return H_min(p_trimmed)


In [6]:
p_example = [0.6, 0.2, 0.15, 0.05]

print("Original distribution:", normalize(p_example))
print("Original H_min =", H_min(p_example))

for eps in [0.0, 0.01, 0.05, 0.1, 0.2]:
    print(f"eps = {eps:0.2f}, H_min_smooth_toy = {H_min_smooth_toy(p_example, eps):0.4f}")


Original distribution: [0.6  0.2  0.15 0.05]
Original H_min = 0.7369655941662062
eps = 0.00, H_min_smooth_toy = 0.7370
eps = 0.01, H_min_smooth_toy = 0.7225
eps = 0.05, H_min_smooth_toy = 0.6630
eps = 0.10, H_min_smooth_toy = 0.5850
eps = 0.20, H_min_smooth_toy = 0.4150


In [7]:
def P_guess(p):
    """Optimal guessing probability: max_x p_x."""
    p = normalize(p)
    return np.max(p)

for p in [p_fair, p_biased, p_four]:
    print("p =", normalize(p))
    print("P_guess =", P_guess(p))
    print("H_min   =", H_min(p))
    print("2^{-H_min} =", 2 ** (-H_min(p)))
    print("-" * 40)


p = [0.5 0.5]
P_guess = 0.5
H_min   = 1.0
2^{-H_min} = 0.5
----------------------------------------
p = [0.9 0.1]
P_guess = 0.9
H_min   = 0.15200309344504995
2^{-H_min} = 0.9
----------------------------------------
p = [0.5 0.2 0.2 0.1]
P_guess = 0.5000000000000001
H_min   = 0.9999999999999997
2^{-H_min} = 0.5000000000000001
----------------------------------------
